In [1]:
import numpy as np
from env.chp import CombinedHeatPowerPlant
from agent.dqn import DoubleQLearner
import csv

Using TensorFlow backend.


In [2]:
EPISODES = 10  # naive episode inclusive
EPISODE_LENGTH = 336  # number of half hour periods per episode
LAG = 1  # positive = see future.  negative = can't see present
RANDOM_TS = True
GEN_OUTPUTS = 100  # generate outputs every n episodes
RUN_NAME = 'EPISODE_LENGTH_' + str(EPISODE_LENGTH)

In [ ]:
env = CombinedHeatPowerPlant(episode_length=EPISODE_LENGTH, lag=LAG, random_ts=RANDOM_TS, verbose=1)

In [ ]:
agent = DoubleQLearner(env, verbose=1, device=0, run_name=RUN_NAME)

Q1 function being created
Q2 function already exists
memory.pickle already exists
network_memory.pickle already exists
info.pickle already exists


In [ ]:
# Running naive episode
episode = 1  # naive policy
agent.policy_ = 0
agent.single_episode(episode)

In [ ]:
# Running e-greedy episode
agent.policy_ = 1  # e-greedy policy
e_greedy_episodes = np.arange(2, EPISODES + 1)
for episode in e_greedy_episodes:
    agent.single_episode(episode)

In [ ]:
# Running greedy episode
episode += 1
agent.epsilon = 0  # greedy
agent.single_episode(episode)

In [ ]:
production_env = CombinedHeatPowerPlant(episode_length=5000, lag=1, random_ts=False, verbose=1)

In [ ]:
len(production_env.actual_state)

In [ ]:
csv_file = open('results_more_info.csv', 'wb')
fieldnames = [
    'Timestamp', 'Settlement period',
    'HGH','LGH','Cooling',
    'Electrical','Ambient temperature',
    'Gas price','Import electricity price','Export electricity price',
    'GT 1 Load', 'GT 2 Load', 'GT 1 Cmd', 'GT 2 Cmd', 'Electricty Generated']
writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

In [ ]:
writer.writeheader()

In [ ]:
production_env.reset()
done = False
index = 0
agent.epsilon = 0
while done is not True:
    state = production_env.state
    action, state_action, choice = agent.policy(state)
    
    # Generate Info Row
    current_state = production_env.state.to_dict()
    current_loads = production_env.asset_states()
    current_state['Index'] = index
    current_state['GT 1 Load'] = current_loads['GT 1'][0]['Status']
    current_state['GT 2 Load'] = current_loads['GT 2'][0]['Status']
    current_state['GT 1 Cmd'] = action[0]
    current_state['GT 2 Cmd'] = action[1]
    current_state['Electricty Generated'] = env_info[-1][1]
    
    writer.writerow(current_state)
    next_state, reward, done, env_info = production_env.step(action)
    index += 1

In [ ]:
production_env.reset()

In [ ]:
action, state_action, choice = agent.policy(state)

In [ ]:
next_state, reward, done, env_info = production_env.step(action)

In [ ]:
env_info